In [1]:
import pandas as pd
import numpy as np
import pickle
import random
from tqdm import tqdm
import pprint

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# npois = 29071
# nspus = 179778

npois = 29924
nusers = 200000
# nspus = 195244

In [2]:
def sparseFeature(feat, feat_num, embed_dim=4):
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}


def denseFeature(feat):
    return {'feat': feat}

In [3]:
datapath = "../data/SMP新数据-20210607-20210702/"
d_session = pd.read_csv(datapath+"orders_poi_session.txt", sep='\t')
d_train = pd.read_csv(datapath+"orders_train.txt", encoding="utf-8", sep='\t')
spu_train=pd.read_csv(datapath+"orders_spu_train.txt", encoding="utf-8", sep='\t')
poi_test = pd.read_csv(datapath + "orders_test_poi.txt", encoding="utf-8", sep='\t')
pois=pd.read_csv(datapath + "pois.txt", encoding="utf-8", sep='\t')
spus=pd.read_csv(datapath + "spus.txt", encoding="utf-8", sep='\t')
users=pd.read_csv(datapath + "users.txt", encoding="utf-8", sep='\t')

In [4]:
users.head()

,user_id,avg_pay_amt,avg_pay_amt_weekdays,avg_pay_amt_weekends
0,0,"[36,49)","[36,49)","[36,49)"
1,1,"[29,36)","[29,36)",<29
2,2,>=65,>=65,>=65
3,3,<29,<29,<29
4,4,"[29,36)","[29,36)",未知


In [5]:
pois.head()

,wm_poi_id,wm_poi_name,primary_second_tag_name,primary_third_tag_name,primary_first_tag_name,poi_brand_id,aor_id,poi_score,delivery_comment_avg_score,food_comment_avg_score
0,0.0,e22575-9c1285-8a1981-ac1915-581787-893564-2915...,0,0,0,0.0,10.0,4.84,5.00,4.86
1,1.0,c91218-f02326-163545-bf1971-49263-952431-78332...,0,0,0,1.0,7.0,4.67,4.81,4.68
2,2.0,1e3493-15577-c93436-ac1915-ca2776-29510-32594-...,1,未知,1,2.0,6.0,4.82,5.00,5.00
3,3.0,1e1273-e22575-002791-a71384-583180-803484-ac19...,0,0,0,3.0,7.0,4.83,4.96,4.84
4,4.0,1e1273-e22575-002791-a71384-583180-803484-ac19...,0,0,0,3.0,5.0,4.91,5.00,4.93


In [6]:
spus.head()

,wm_food_spu_id,wm_food_spu_name,price,category,ingredients,taste,stand_food_id,stand_food_name
0,0,833366-bf3929-254096,26.0,[0],"[0, 1, 2]",[0],0.0,833366-bf3929-254096
1,1,032521-a0772-3f3352,20.0,[1],"[3, 1]",[1],1.0,032521-a0772-3f3352
2,2,952431-c52358-c22123-7f760-832660-fe1862,18.0,[2],"[3, 1, 4, 5, 6, 7]",[2],2.0,952431-c52358-c22123-7f760-832660
3,3,e84149-6b1215-ca2776-441372-b5649-ac1915-f4338...,10.0,[3],"[8, 9]",[0],3.0,e84149-6b1215-ca2776-441372
4,4,6b1215-c9725-862243-102951-1f1116,3.58,[4],NaN,NaN,NaN,NaN


In [7]:
d_session.head()

,wm_order_id,clicks,dt
0,1103,2168#1884,20210607
1,24171,NaN,20210607
2,27913,1371#1494#3518,20210607
3,24875,1901#1901#1901#1901#2398,20210607
4,23149,NaN,20210607


In [8]:
d_train.head()

,user_id,wm_order_id,wm_poi_id,aor_id,order_price_interval,order_timestamp,ord_period_name,order_scene_name,aoi_id,takedlvr_aoi_type_name,dt
0,178557,0,2334,6,<29,1623061539,3,0,NaN,未知,20210607
1,175118,1,3315,0,<29,1623032193,1,1,0.0,0,20210607
2,36208,2,2168,0,<29,1623036350,1,0,1.0,0,20210607
3,102798,3,3071,0,"[29,36)",1623071723,4,0,2.0,0,20210607
4,73712,4,2902,0,"[49,65)",1623020472,0,2,3.0,1,20210607


# 商家分析

In [158]:
pois.head()

,wm_poi_id,wm_poi_name,primary_second_tag_name,primary_third_tag_name,primary_first_tag_name,poi_brand_id,aor_id,poi_score,delivery_comment_avg_score,food_comment_avg_score
0,0.0,e22575-9c1285-8a1981-ac1915-581787-893564-2915...,0,0,0,0.0,10.0,4.84,5.00,4.86
1,1.0,c91218-f02326-163545-bf1971-49263-952431-78332...,0,0,0,1.0,7.0,4.67,4.81,4.68
2,2.0,1e3493-15577-c93436-ac1915-ca2776-29510-32594-...,1,未知,1,2.0,6.0,4.82,5.00,5.00
3,3.0,1e1273-e22575-002791-a71384-583180-803484-ac19...,0,0,0,3.0,7.0,4.83,4.96,4.84
4,4.0,1e1273-e22575-002791-a71384-583180-803484-ac19...,0,0,0,3.0,5.0,4.91,5.00,4.93


In [9]:
pois.describe()

,wm_poi_id,poi_brand_id,aor_id,poi_score,delivery_comment_avg_score,food_comment_avg_score
count,29924.000000,19411.000000,28401.000000,27892.000000,27892.000000,27892.000000
mean,14961.500000,153.902272,207.730749,3.913871,4.100537,3.475916
std,8638.459064,278.167496,400.040860,1.740303,1.816807,2.054673
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7480.750000,1.000000,19.000000,4.370000,4.760000,0.000000
50%,14961.500000,1.000000,68.000000,4.680000,4.910000,4.640000
75%,22442.250000,180.500000,169.000000,4.820000,5.000000,4.800000
max,29923.000000,1352.000000,2285.000000,5.000000,5.000000,5.000000


商家编号从0到29923共29924个。

In [10]:
pois.isna().sum()

wm_poi_id                         1
wm_poi_name                    1524
primary_second_tag_name        1524
primary_third_tag_name         1524
primary_first_tag_name         1524
poi_brand_id                  10514
aor_id                         1524
poi_score                      2033
delivery_comment_avg_score     2033
food_comment_avg_score         2033
dtype: int64

In [11]:
pois.loc[pois["wm_poi_id"].isna()]

,wm_poi_id,wm_poi_name,primary_second_tag_name,primary_third_tag_name,primary_first_tag_name,poi_brand_id,aor_id,poi_score,delivery_comment_avg_score,food_comment_avg_score
29924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


共计1353个品牌。

In [12]:
pois.drop(index=29924,inplace=True)

注意到很多商家并没有品牌。需要给它们补一类空品牌。

In [13]:
pprint.pprint(pois.poi_brand_id.unique())

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.350e+03, 1.351e+03,
       1.352e+03])


In [14]:
pois.poi_brand_id.fillna(value=-1,inplace=True)

In [15]:
pois.describe()

,wm_poi_id,poi_brand_id,aor_id,poi_score,delivery_comment_avg_score,food_comment_avg_score
count,29924.000000,29924.000000,28401.000000,27892.000000,27892.000000,27892.000000
mean,14961.500000,99.481486,207.730749,3.913871,4.100537,3.475916
std,8638.459064,235.924373,400.040860,1.740303,1.816807,2.054673
min,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,7480.750000,-1.000000,19.000000,4.370000,4.760000,0.000000
50%,14961.500000,1.000000,68.000000,4.680000,4.910000,4.640000
75%,22442.250000,30.000000,169.000000,4.820000,5.000000,4.800000
max,29923.000000,1352.000000,2285.000000,5.000000,5.000000,5.000000


注意到一个有意思的现象，这几个的空值都是一致的。
```
wm_poi_name                    1524
primary_second_tag_name        1524
primary_third_tag_name         1524
primary_first_tag_name         1524
aor_id                         1524
```

In [17]:
pois.loc[pois.wm_poi_name.isna()].count()

wm_poi_id                     1523
wm_poi_name                      0
primary_second_tag_name          0
primary_third_tag_name           0
primary_first_tag_name           0
poi_brand_id                  1523
aor_id                           0
poi_score                        0
delivery_comment_avg_score       0
food_comment_avg_score           0
dtype: int64

In [19]:
pois.loc[pois.wm_poi_name.isna()].head()

,wm_poi_id,wm_poi_name,primary_second_tag_name,primary_third_tag_name,primary_first_tag_name,poi_brand_id,aor_id,poi_score,delivery_comment_avg_score,food_comment_avg_score
4647,4647.0,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
4656,4656.0,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
4659,4659.0,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
4660,4660.0,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
4662,4662.0,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN


这个说明所有没有名字的商家这些所有分数都是空的，可能是无效商家。这里建立一个它们的poi_id方便查阅后面是否有人在这些商家中买过东西。

In [22]:
pois.shape

(28401, 10)

In [24]:
null_pois=set(pois.loc[pois.wm_poi_name.isna()]["wm_poi_id"])
print(len(null_pois))
#为了方便分析去掉这些空商家
pois.drop(index=pois.loc[pois.wm_poi_name.isna()].index,inplace=True)

0


In [23]:
pois.loc[pois["poi_score"].isna()]

,wm_poi_id,wm_poi_name,primary_second_tag_name,primary_third_tag_name,primary_first_tag_name,poi_brand_id,aor_id,poi_score,delivery_comment_avg_score,food_comment_avg_score
169,169.0,e91057-e91057-bb1534-ac1915-ca861-921507-1a278...,5,16,0,1.0,8.0,NaN,NaN,NaN
339,339.0,1c1140-0548-a12982-9e2508-5b2605-bb1073-934253...,14,未知,1,1.0,8.0,NaN,NaN,NaN
517,517.0,5e2642-d2176-072548-441623-952431-af38-b01235-...,5,16,0,1.0,5.0,NaN,NaN,NaN
703,703.0,242306-313920-ae3347-08281-f6169-d82402,2,9,0,1.0,8.0,NaN,NaN,NaN
717,717.0,533719-a4877-503519-934253-152609-ac1915-f5300...,14,未知,1,1.0,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
29907,29907.0,423774-a34234-8c660-752587-733154-ac1915-82255...,35,未知,7,-1.0,79.0,NaN,NaN,NaN
29908,29908.0,a21409-2a3910-772036-c93436-a61030-461031-da3326,2,4,0,-1.0,2283.0,NaN,NaN,NaN
29912,29912.0,a9518-e71036-6d3546-aa25-b1239-a34031-354111-3...,1,未知,1,1.0,71.0,NaN,NaN,NaN
29913,29913.0,f81995-ae620-e11431-892342-ac1915-3d3353-9a366...,9,未知,1,-1.0,0.0,NaN,NaN,NaN


In [170]:
def for_print(x):
    try:
        int(x)
    except:
        return x
    else:
        return None
pois[["primary_second_tag_name","primary_third_tag_name","primary_first_tag_name"]].applymap(for_print).apply(pd.unique)

,primary_second_tag_name,primary_third_tag_name,primary_first_tag_name
0,None,None,None
1,未知,未知,未知


这里看到一个很有意思的现象，"primary_second_tag_name","primary_third_tag_name","primary_first_tag_name"三个值不是空叫未知。

有些商家并没有评分。可能是新店。
给它们加一列标志出来。为了方便处理将所有这类店的评分补成最低分。

In [25]:
def sign_for_no_score(x):
    return 1 if pd.isna(x) else 0
pois["is_null"]=pois["poi_score"].map(sign_for_no_score)

In [26]:
for col in ["poi_score","delivery_comment_avg_score","food_comment_avg_score"]:
    pois[col].fillna(value=0,inplace=True)

In [27]:
pois.isna().sum()

wm_poi_id                     0
wm_poi_name                   0
primary_second_tag_name       0
primary_third_tag_name        0
primary_first_tag_name        0
poi_brand_id                  0
aor_id                        0
poi_score                     0
delivery_comment_avg_score    0
food_comment_avg_score        0
is_null                       0
dtype: int64

空值处理完成。

# 菜品处理

In [29]:
spus.head()

,wm_food_spu_id,wm_food_spu_name,price,category,ingredients,taste,stand_food_id,stand_food_name
0,0,833366-bf3929-254096,26.0,[0],"[0, 1, 2]",[0],0.0,833366-bf3929-254096
1,1,032521-a0772-3f3352,20.0,[1],"[3, 1]",[1],1.0,032521-a0772-3f3352
2,2,952431-c52358-c22123-7f760-832660-fe1862,18.0,[2],"[3, 1, 4, 5, 6, 7]",[2],2.0,952431-c52358-c22123-7f760-832660
3,3,e84149-6b1215-ca2776-441372-b5649-ac1915-f4338...,10.0,[3],"[8, 9]",[0],3.0,e84149-6b1215-ca2776-441372
4,4,6b1215-c9725-862243-102951-1f1116,3.58,[4],NaN,NaN,NaN,NaN


In [30]:
spus.groupby("wm_food_spu_name").count()

,wm_food_spu_id,price,category,ingredients,taste,stand_food_id,stand_food_name
wm_food_spu_name,,,,,,,
001573-0d1248-f81995-7b3000-ca2776-e24016-fe1905-001573-0d1248-f81995-69458-952779-c52358-e24016-fe1905-492276-461622-3f3352,1,1,1,1,0,0,0
001573-0d1248-f81995-952431-273904-e24016-fe1905-001573-0d1248-f81995-3e439-1a2204-e24016-fe1905-f53582-581804-291145-ea4014-bb1073,1,1,1,1,0,0,0
001573-0d1248-f81995-952431-273904-e24016-fe1905-ff1797-b02457-4e2941-053997-002791-752878,1,1,1,1,0,0,0
001573-0d1248-f81995-f53582-ee245-952431-b01235-e24016-ac1915-ec555-1f1116-4b1465,1,1,1,0,1,1,1
001573-14244-e91936-bf1971,6,6,6,0,4,0,0
...,...,...,...,...,...,...,...
ff88-ed2459-321201-c42819-97564-7f760-952431-273904-b01235-e53235-b71967-9e3533-482396-ac1915-e13049-362686-c62039-88996-611016-c9725-254096-d34075-4b1465-9c3536-d21564,1,1,1,1,0,0,0
ff88-f52772-202191-62745-c3430-984171-ac1915-a73166-9e1412-e1212-3c2046-4b1465,1,1,1,1,0,0,0
ff88-f52772-202191-ad1006-fa1918-1e3493-952431-582862-ac1915-31159-611016-c9725-254096-4b1465,1,1,1,1,1,1,1


菜品名字有相同，说明多个店铺可能有相同菜名。

In [31]:
spus.isna().sum()

wm_food_spu_id           0
wm_food_spu_name      7506
price                    6
category              9281
ingredients          78642
taste               129495
stand_food_id        91750
stand_food_name      91768
dtype: int64

In [179]:
spu_no_name_ids=spus.loc[spus["wm_food_spu_name"].isna()]["wm_food_spu_id"]

In [185]:
spus.loc[spus["wm_food_spu_name"].isna()].isna().sum()

wm_food_spu_id         0
wm_food_spu_name    7506
price                  1
category            7506
ingredients         7506
taste               7506
stand_food_id       7505
stand_food_name     7506
dtype: int64

In [33]:
spu_no_name_ids=spus.loc[spus["price"].isna()]["wm_food_spu_id"]

现在单独来分析下这部分数据在订单中有没有被人买过.

In [34]:
all_used_poi=set(d_train["wm_poi_id"].tolist())
all_used_spu=set(spu_train["wm_food_spu_id"].tolist())
count=0
for i in spu_no_name_ids:
    if i in all_used_spu:
        count+=1
print(count)
print("below is spu\n------\nnow is poi:")
for i in null_pois:
    if i in all_used_poi:
        print(str(i)+" ")

6
below is spu
------
now is poi:


```
wm_food_spu_id	wm_food_spu_name	price	category	ingredients	taste	stand_food_id	stand_food_name
```
以上是菜品的信息，其它特征不好处理目前只保留价格和id，并且价格中的空值用平均值填补。价格中有个奇怪的地方,有几个菜品价格放错了。

In [35]:
lajidongxi=[]
for j,i in enumerate(spus["price"]):
    try:
        float(i)
    except:
        lajidongxi.append(j)
spus.iloc[lajidongxi,:]

,wm_food_spu_id,wm_food_spu_name,price,category,ingredients,taste,stand_food_id,stand_food_name
4276,4276,b63994-181136-62745-621385,"[""芥末"",""酱油"",""三文鱼"",""北极贝""]",[267],[553],[41],1520.0,NaN
7723,7723,f53921-974186-5f1012-c72464,"[""香油"",""盐"",""醋"",""里脊"",""淀粉"",""料酒"",""酱油""]",NaN,[642],[41],1520.0,NaN
17167,17167,f0118-1e3493-d41611-ca861-57325-921507-4e2941-...,"[""金枪鱼""]",[267],[642],[45],1520.0,NaN
17638,17638,001573-b3305-502832-7c693-49263-6b2866-6b2866-...,"[""甜味""]",[311],[642],[41],1520.0,NaN
92178,92178,ac1915-783954-4b1465-5b526-5b526-1d2891-a4404-...,"[""核桃""]",NaN,NaN,NaN,1520.0,NaN
124005,124005,47682-d44105-9e3533-b01235-be2234,"[""洋葱"",""料酒"",""红椒"",""孜然""]",NaN,NaN,NaN,1520.0,NaN
134805,134805,f53921-974186-5f1012-c72464,"[""香油"",""盐"",""醋"",""里脊"",""淀粉"",""料酒"",""酱油""]",NaN,[642],[41],1520.0,NaN
135264,135264,47682-d44105-9e3533-b01235-be2234,"[""洋葱"",""料酒"",""红椒"",""孜然""]",NaN,NaN,NaN,1520.0,NaN
140209,140209,251180-863514-e11431-e24016,"[""咖啡"",""奶酪"",""奶油"",""细砂糖""]",[357],[642],[41],1520.0,NaN
142431,142431,4b519-cf981-4b519-92207-62745-c9725-cf3692-136...,"[""小米""]",[267],[642],[41],1520.0,NaN


In [36]:


nsum=0
count=0
for i in spus["price"]:
    try:
        float(i)
    except:
        pass
    else:
        nsum+=float(i)
        count+=1
avg=nsum/count
spus["price"].fillna(avg,inplace=True)
for i in lajidongxi:
    spus.loc[i,"price"]=avg
spus["price"]=spus["price"].astype("float")

In [221]:
spus.loc[spus["price"].isna()]

,wm_food_spu_id,wm_food_spu_name,price,category,ingredients,taste,stand_food_id,stand_food_name
4276,4276,b63994-181136-62745-621385,NaN,[267],[553],[41],1520.0,NaN
7723,7723,f53921-974186-5f1012-c72464,NaN,NaN,[642],[41],1520.0,NaN
17167,17167,f0118-1e3493-d41611-ca861-57325-921507-4e2941-...,NaN,[267],[642],[45],1520.0,NaN
17638,17638,001573-b3305-502832-7c693-49263-6b2866-6b2866-...,NaN,[311],[642],[41],1520.0,NaN
19595,19595,f0118-1e3493-934253-7b3000-cd2028-a61030-4e294...,NaN,NaN,NaN,NaN,1520.0,NaN
29400,29400,d24190-393429-14244-561292-2f3129-c11501-9c353...,NaN,NaN,NaN,NaN,1520.0,NaN
63095,63095,NaN,NaN,NaN,NaN,NaN,1520.0,NaN
92178,92178,ac1915-783954-4b1465-5b526-5b526-1d2891-a4404-...,NaN,NaN,NaN,NaN,1520.0,NaN
103740,103740,bb1073-d53544-b5649-ac1915-9e1412-1f1116-4b1465,NaN,NaN,NaN,NaN,1520.0,NaN
105559,105559,f0118-1e3493-743101-952431-b01235-4e2941-05399...,NaN,[267],[642],[41],1520.0,NaN


In [37]:
spus[["wm_food_spu_id","price"]]

,wm_food_spu_id,price
0,0,26.00
1,1,20.00
2,2,18.00
3,3,10.00
4,4,3.58
...,...,...
195239,195239,19.00
195240,195240,8.90
195241,195241,42.00
195242,195242,3.00


# 菜品和商家联合处理成商家特征

## 商铺特色菜品
商铺中售出最受欢迎的前3个菜品
菜品spu跟orders_spu_tain on wm_food_spu_id, 然后on wm_order_id跟orders_train,然后on wm_poi_id跟pois

按商家分类，然后按菜品分类，查看菜品订单有几个.



In [38]:
all_table=pd.merge(pd.merge(pd.merge(spu_train,spus,on="wm_food_spu_id"),d_train,on="wm_order_id"),pois,on="wm_poi_id")

In [39]:
all_table.head()

,wm_order_id,wm_food_spu_id,dt_x,wm_food_spu_name,price,category,ingredients,taste,stand_food_id,stand_food_name,...,wm_poi_name,primary_second_tag_name,primary_third_tag_name,primary_first_tag_name,poi_brand_id,aor_id_y,poi_score,delivery_comment_avg_score,food_comment_avg_score,is_null
0,10543,159611,20210607,87237-2d165-561292-08857-3f3352-3f3352-302095-...,1.88,[116],NaN,[4],NaN,NaN,...,842893-791694-14244-561292-2f3129-ac1915-f5300...,0,10,0,14.0,7.0,4.51,4.89,4.57,0
1,10543,160913,20210607,ab3097-ae1935-9c3536-d21564,32.80,[40],NaN,NaN,NaN,NaN,...,842893-791694-14244-561292-2f3129-ac1915-f5300...,0,10,0,14.0,7.0,4.51,4.89,4.57,0
2,10543,187248,20210607,ac1915-783954-162486-4b1465-072795-f8226-ac191...,4.58,[176],NaN,[1],2614.0,072795-f8226,...,842893-791694-14244-561292-2f3129-ac1915-f5300...,0,10,0,14.0,7.0,4.51,4.89,4.57,0
3,34899,159611,20210607,87237-2d165-561292-08857-3f3352-3f3352-302095-...,1.88,[116],NaN,[4],NaN,NaN,...,842893-791694-14244-561292-2f3129-ac1915-f5300...,0,10,0,14.0,7.0,4.51,4.89,4.57,0
4,34899,117109,20210607,001573-e91936-984171-ac1915-9e1412-1f1116-4b1465,4.58,[5],NaN,NaN,NaN,NaN,...,842893-791694-14244-561292-2f3129-ac1915-f5300...,0,10,0,14.0,7.0,4.51,4.89,4.57,0


In [40]:
special_spu=all_table.groupby(["wm_poi_id","wm_food_spu_id"]).count()["wm_order_id"].reset_index()

In [41]:
special_spu

,wm_poi_id,wm_food_spu_id,wm_order_id
0,0,1105,1
1,0,3587,1
2,0,8129,8
3,0,8362,3
4,0,9690,34
...,...,...,...
182748,3576,139961,1
182749,3576,154459,1
182750,3576,164722,1
182751,3576,170004,2


In [ ]:
# special_spu.hist(column=)

In [42]:
special_spu.quantile(0.9)

wm_poi_id           3219.0
wm_food_spu_id    175743.8
wm_order_id           41.0
Name: 0.9, dtype: float64

In [275]:
special_spu.loc[special_spu["wm_poi_id"]==0].sort_values('wm_order_id',ascending=False)

,wm_poi_id,wm_food_spu_id,wm_order_id
8193,0,159409,79
10501,0,24821,65
11961,0,169512,58
13577,0,68258,53
13851,0,190495,52
...,...,...,...
162151,0,37768,1
162150,0,34135,1
162149,0,129140,1
151856,0,1105,1


In [43]:
K=3
spe_spu=special_spu.groupby("wm_poi_id").apply(lambda x:x.sort_values('wm_order_id',ascending=False)['wm_food_spu_id'].head(K).values)

In [44]:
spe_spu

wm_poi_id
0       [159409, 24821, 169512]
1       [32717, 129549, 121022]
2         [94721, 48478, 32729]
3        [110132, 116759, 1421]
4         [36051, 120427, 8999]
                 ...           
3572      [138417, 9866, 94333]
3573    [90308, 138420, 140208]
3574      [1615, 63990, 111658]
3575      [16985, 69249, 29251]
3576     [4406, 170004, 137616]
Length: 3526, dtype: object

In [45]:
spus.loc[spus["price"].isna()]

,wm_food_spu_id,wm_food_spu_name,price,category,ingredients,taste,stand_food_id,stand_food_name
4276,4276,b63994-181136-62745-621385,NaN,[267],[553],[41],1520.0,NaN
7723,7723,f53921-974186-5f1012-c72464,NaN,NaN,[642],[41],1520.0,NaN
17167,17167,f0118-1e3493-d41611-ca861-57325-921507-4e2941-...,NaN,[267],[642],[45],1520.0,NaN
17638,17638,001573-b3305-502832-7c693-49263-6b2866-6b2866-...,NaN,[311],[642],[41],1520.0,NaN
19595,19595,f0118-1e3493-934253-7b3000-cd2028-a61030-4e294...,NaN,NaN,NaN,NaN,1520.0,NaN
29400,29400,d24190-393429-14244-561292-2f3129-c11501-9c353...,NaN,NaN,NaN,NaN,1520.0,NaN
63095,63095,NaN,NaN,NaN,NaN,NaN,1520.0,NaN
92178,92178,ac1915-783954-4b1465-5b526-5b526-1d2891-a4404-...,NaN,NaN,NaN,NaN,1520.0,NaN
103740,103740,bb1073-d53544-b5649-ac1915-9e1412-1f1116-4b1465,NaN,NaN,NaN,NaN,1520.0,NaN
105559,105559,f0118-1e3493-743101-952431-b01235-4e2941-05399...,NaN,[267],[642],[41],1520.0,NaN


## 商家菜品数量，热门菜数量

In [294]:
poi_spus_num=special_spu.groupby("wm_poi_id").count()["wm_food_spu_id"]

In [307]:
spe_threshold=41
poi_spe_spus_num=special_spu.groupby("wm_poi_id").apply(lambda x:x['wm_order_id']>spe_threshold).groupby("wm_poi_id").sum()

wm_poi_id
0       13
1        3
2        2
3       14
4       26
        ..
3572     0
3573     3
3574     0
3575     2
3576     0
Name: wm_order_id, Length: 3526, dtype: int64

## POI 的热度

In [315]:
poi_order=all_table[['wm_order_id','wm_poi_id']].groupby("wm_poi_id").count()

,wm_order_id
wm_poi_id,
0,1181
1,571
2,301
3,1797
4,2787
...,...
3572,38
3573,370
3574,6
